<a href="https://colab.research.google.com/github/sainijagjit/CS6910-A2/blob/main/Assignment_2_iNaturalist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
! pip install pytorch-lightning --quiet
# install weights and biases
!pip install wandb --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.9/264.9 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00


In [65]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as L
from torch.utils.data import DataLoader, random_split, SubsetRandomSampler
from sklearn.model_selection import train_test_split
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import os
import matplotlib.pyplot as plt
import numpy as np
import torchvision
from torchmetrics import Accuracy
from pytorch_lightning.callbacks import EarlyStopping,Callback,ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
import wandb

In [66]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Deep Learning/inaturalist_12K/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Deep Learning/inaturalist_12K


In [67]:
wandb.login(key="6175f01f348073c42fd3c8f1943549684e0e1fc0")
wandb_project="CS6910 - Assignment 2"
wandb_entity="ns24z060"

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


**Loading & Splitting Dataset**

In [21]:
transform = transforms.Compose(
    [ transforms.Resize((224, 224)),
      transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = ImageFolder(root='train', transform=transform)
input_dims = train_dataset[0][0].size()
print(f'Input image shape ({input_dims[0]},{input_dims[1]},{input_dims[2]})')
print('-------------')

train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size


train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])
test_dataset = ImageFolder(root='val',transform=transform)


class_names = train_dataset.dataset.classes
class_counts = {class_name: 0 for class_name in class_names}
for _, class_idx in train_dataset.dataset.samples:
    class_name = class_names[class_idx]
    class_counts[class_name] += 1

for class_name, count in class_counts.items():
    print(f"Class: {class_name}, No. of samples: {count}")

num_classes = len(class_counts.keys())
print('-------------')
print(f'No of classes {num_classes}')
print(f'Train Samples {len(train_dataset)}')
print(f'Val Samples {len(val_dataset)}')
print(f'Test Samples {len(test_dataset)}')

Input image shape (3,224,224)
-------------
Class: Amphibia, No. of samples: 1010
Class: Animalia, No. of samples: 1000
Class: Arachnida, No. of samples: 1001
Class: Aves, No. of samples: 1000
Class: Fungi, No. of samples: 999
Class: Insecta, No. of samples: 1010
Class: Mammalia, No. of samples: 1000
Class: Mollusca, No. of samples: 1010
Class: Plantae, No. of samples: 1000
Class: Reptilia, No. of samples: 1022
-------------
No of classes 10
Train Samples 8041
Val Samples 2011
Test Samples 2001


In [15]:
batch_size=8
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# def imshow(img):
#     img = img / 2 + 0.5
#     npimg = img.numpy()
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))
#     plt.show()

# dataiter = iter(train_dataloader)
# images, labels = next(dataiter)


# imshow(torchvision.utils.make_grid(images))

In [59]:
class Net(nn.Module):
  def __init__(self,
                cnn_activation_function,
                fcc_activation_function,
                num_filters,
                filter_size,
                dense_neurons,
                stride=2,
                input_dims=input_dims,
                num_classes=num_classes):
      super().__init__()
      cnn_layers = []
      for i in range(5):
        in_channels = input_dims[0] if i==0 else num_filters
        cnn_layers.extend([nn.Conv2d(in_channels, num_filters, kernel_size=filter_size),
                      getattr(nn, cnn_activation_function)(inplace=True),
                      nn.MaxPool2d(kernel_size=filter_size,stride=stride),
                      ])
      self.cnn_net = nn.Sequential(*cnn_layers)


      n_sizes = self._get_conv_output(input_dims)
      dnn_layers = [
          nn.Flatten(),
          nn.Linear(n_sizes, dense_neurons),
          getattr(nn, fcc_activation_function)(inplace=True),
          nn.Linear(dense_neurons, num_classes)]
      self.dnn_net = nn.Sequential(*dnn_layers)

  def _get_conv_output(self, shape):
        input = torch.autograd.Variable(torch.rand(batch_size, *shape))
        output_feat = self._forward_features(input)
        n_size = output_feat.data.view(batch_size, -1).size(1)
        return n_size

  def _forward_features(self, x):
        return self.cnn_net(x)

  def forward(self, x):
        x =  self.cnn_net(x)
        x =  self.dnn_net(x)
        return F.log_softmax(x, dim=1)

In [62]:
class MyCnnModel(L.LightningModule):
    def __init__(self,
                 cnn_activation_function = 'ReLU',
                 fcc_activation_function = 'ReLU',
                 loss_function='cross_entropy',
                 num_filters=32,
                 filter_size=3,
                 dense_neurons=1024,
                 learning_rate=0.01,
                 optimizer='Adam'):
        super().__init__()
        self.save_hyperparameters()
        self.model = Net(cnn_activation_function,fcc_activation_function,num_filters,filter_size,dense_neurons)
        self.accuracy = Accuracy(task="multiclass", num_classes=num_classes)
        self.loss_function = getattr(F, loss_function)
        self.learning_rate = learning_rate
        self.optimizer = optimizer

    def forward(self, x):
        x = self.model(x)
        return F.softmax(x, dim=1)

    def training_step(self, batch, batch_idx):
      loss,acc = self._common_step(batch,batch_idx)
      self.log('train_loss', loss, on_step=True, on_epoch=True, logger=True)
      self.log('train_acc', acc, on_step=True, on_epoch=True, logger=True)
      return loss

    def validation_step(self, batch, batch_idx):
      loss,acc =  self._common_step(batch,batch_idx)
      self.log('val_loss', loss, prog_bar=True)
      self.log('val_acc', acc, prog_bar=True)
      return loss


    def test_step(self,batch,batch_idx):
      loss,acc =  self._common_step(batch,batch_idx)
      self.log('test_loss', loss, prog_bar=True)
      self.log('test_acc', acc, prog_bar=True)
      return loss

    def _common_step(self, batch, batch_idx):
      inputs, target = batch
      output = self.forward(inputs)
      loss = self.loss_function(output, target)
      acc = self.accuracy(output, target)
      return loss,acc

    def configure_optimizers(self):
      return getattr(torch.optim,self.optimizer)(self.parameters(), lr=self.learning_rate)

In [ ]:
class PrintCallback(Callback):
    def on_train_start(self, trainer, pl_module):
        print("Training is started!")
    def on_train_end(self, trainer, pl_module):
        print("Training is done.")


model = MyCnnModel()
checkpoint_callback = ModelCheckpoint(monitor="val_loss", mode='max')
wandb_logger = WandbLogger(project=wandb_project, job_type='train', log_model='all')
early_stop_callback = EarlyStopping(monitor="val_loss")

trainer = L.Trainer(
    max_epochs=1,
    limit_train_batches=0.1,
    limit_val_batches=0.1,
    check_val_every_n_epoch=10,
    log_every_n_steps=1,
    logger=wandb_logger,
    # fast_dev_run=1,
    callbacks=[PrintCallback(),checkpoint_callback,early_stop_callback])
trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)
wandb.finish()

INFO:pytorch_lightning.utilities.rank_zero:Using bfloat16 Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


INFO:pytorch_lightning.callbacks.model_summary:
  | Name     | Type               | Params
------------------------------------------------
0 | model    | Net                | 573 K 
1 | accuracy | MulticlassAccuracy | 0     
------------------------------------------------
573 K     Trainable params
0         Non-trainable params
573 K     Total params
2.294     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training is started!


Training: |          | 0/? [00:00<?, ?it/s]

In [14]:
trainer.test(model, dataloaders=test_dataloader, verbose=False)

Testing: |          | 0/? [00:00<?, ?it/s]